In [1]:
import pandas as pd
import glob
import re
from tqdm import tqdm

In [2]:
#!pip install openpyxl
#Use this if .xls not working

# import win32com.client as win32
# fname = 'D:\\졸논\\FCT_WRN_20210308015339.xls'
# excel = win32.gencache.EnsureDispatch('Excel.Application')
# print('ok..1')
# wb = excel.Workbooks.Open(fname)
# print('ok..2')
# wb.SaveAs(fname+"x",FileFormat = 51)
# print('ok..3')
# wb.Close()
# print('ok..4')
# excel.Application.Quit()
# print('ok..5')
# del excel

In [3]:
data_loc = glob.glob(r'..//Dam_weather_warning//*.xlsx')

In [4]:
j = 0
for data in data_loc:
    df = pd.read_excel(data)
    
    tmp = df['특보통보'].values.tolist()
    if j == 0:
        #발표시각
        announced_time_idx = [i+1 for i,v in enumerate(tmp) if v == '발표시각']
        announced_time = [tmp[i] for i in announced_time_idx]
        print('announced_time (i=0)',len(announced_time))
        #발효시간
        Effective_time_idx = [i+1 for i,v in enumerate(tmp) if v == '발효시각']
        Effective_time = [tmp[i] for i in Effective_time_idx]
        print('Effective_time (i=0)',len(Effective_time))
        #해당지역
        Affected_Location_idx = [i+1 for i,v in enumerate(tmp) if v == '해당지역']
        Affected_Location = [tmp[i] for i in Affected_Location_idx]
        print('Affected_Location (i=0)',len(Affected_Location))
#         #내용
#         Content_idx = [i+1 for i,v in enumerate(tmp) if v == '내용']
#         content = [tmp[i] for i in Content_idx]
#         print('content (i=0)',len(content))
        j+=1
    else:
        announced_time_idx = [i+1 for i,v in enumerate(tmp) if v == '발표시각']
        announced_time.extend([tmp[i] for i in announced_time_idx])
        print(f'announced_time i={j}',len(announced_time))
        
        Effective_time_idx = [i+1 for i,v in enumerate(tmp) if v == '발효시각']
        Effective_time.extend([tmp[i] for i in Effective_time_idx])
        print(f'Effective_time i={j}',len(Effective_time))
        
        Affected_Location_idx = [i+1 for i,v in enumerate(tmp) if v == '해당지역']
        Affected_Location.extend([tmp[i] for i in Affected_Location_idx])
        print(f'Affected_Location i={j}',len(Affected_Location))
        
#         Content_idx = [i+1 for i,v in enumerate(tmp) if v == '내용']
#         content.extend([tmp[i] for i in Content_idx])
#         print(f'content i={i}',len(content))
        
        j += 1

announced_time (i=0) 2554
Effective_time (i=0) 2554
Affected_Location (i=0) 2554
announced_time i=1 7493
Effective_time i=1 7493
Affected_Location i=1 7493
announced_time i=2 12700
Effective_time i=2 12700
Affected_Location i=2 12700


In [5]:
useful_info = {'Time_a':announced_time,#발표시각
              'Time_e':Effective_time, #발효시각
              'Location':Affected_Location#,#해당지역
              }#'content':content}#내용
useful = pd.DataFrame.from_dict(useful_info)
print(useful.shape)

(12700, 3)


In [6]:
#Heavy Rain
def RAIN_ADV(row):
    location = ['춘천','영서','중서']
    tt2 = re.split('(\d)',row['Location'])
    for t in tt2:
        if (('호우경보'in t)&(any(l for l in location if l in t))): 
            return 1
            break
        else:
            pass
    return 0

In [7]:
"""
Extract only heavy rain warning from Chun-Cheon City, Young-seo region, and mid-west area
"""
useful['호우경보'] = useful.apply(lambda row: RAIN_ADV(row), axis=1)

In [8]:
useful = useful[useful['호우경보'] != 0]

In [9]:
useful

,Time_a,Time_e,Location,호우경보
38,2004-07-12 23:00:00,(1) 호우경보 : 2004년 07월 12일 23시 00분,"(1) 호우경보 : 강원도(속초시, 고성군, 양양군, 철원군, 화천군, 홍천군, 춘...",1
39,2004-07-13 09:00:00,(1) 호우경보 : 2004년 07월 13일 09시 00분(2) 호우주의보 : 20...,"(1) 호우경보 : 강원도(속초시, 고성군, 양양군, 철원군, 화천군, 홍천군, 춘...",1
346,2005-07-28 13:00:00,(1) 호우경보 대치 : 2005년 07월 28일 13시 00분,(1) 호우경보 대치 : 강원도(춘천시),1
348,2005-07-28 16:30:00,(1) 호우경보 해제 : 2005년 07월 28일 17시 00분(2) 호우주의보 해...,"(1) 호우경보 해제 : 강원도(춘천시)(2) 호우주의보 해제 : 강원도(고성군, ...",1
699,2006-07-12 11:00:00,(1) 호우경보 대치 : 2006년 07월 12일 11시 00분,"(1) 호우경보 대치 : 강원도(횡성군, 철원군, 화천군, 홍천군, 춘천시, 양구군...",1
...,...,...,...,...
11995,2018-08-28 19:50:00,(1) 호우경보 대치 : 2018년 08월 28일 19시 50분,(1) 호우경보 대치 : 강원도(춘천),1
12007,2018-08-29 05:30:00,(1) 호우경보 해제 : 2018년 08월 29일 05시 30분,(1) 호우경보 해제 : 강원도(춘천),1
12010,2018-08-29 09:10:00,(1) 호우경보 대치 : 2018년 08월 29일 09시 10분,(1) 호우경보 대치 : 강원도(춘천),1
12014,2018-08-30 04:00:00,(1) 호우경보 해제 : 2018년 08월 30일 04시 00분,"(1) 호우경보 해제 : 강원도(강원북부산지, 양구평지, 인제평지, 춘천, 화천, 철원)",1


In [10]:
useful.drop(['Time_e','Location'],inplace=True,axis=1)

In [11]:
useful.to_csv('../Dam_weather_warning/HeavyRainWrn.csv',encoding='euc-kr')